In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/checkin.json
/kaggle/input/yelp-dataset/Dataset_Challenge_Dataset_Agreement.pdf
/kaggle/input/yelp-dataset/Yelp_Dataset_Challenge_Round_13.pdf
/kaggle/input/yelp-dataset/business.json
/kaggle/input/yelp-dataset/tip.json
/kaggle/input/yelp-dataset/user.json
/kaggle/input/yelp-dataset/review.json
/kaggle/input/yelp-dataset/photo.json


## install pyspark
## & import some liberys 

In [2]:
!pip install -q pyspark==3.4.1 spark-nlp==5.1.2
!pip install -q spark-nlp-display

In [3]:
from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col

In [4]:
import pyspark
from pyspark.sql.functions import when
from pyspark.sql import SparkSession

In [5]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col

import pandas as pd

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f83f9323-f4af-44f2-8145-1d3ca5fc10d7;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failureaccess;1.0.1 

Spark NLP version 5.1.2
Apache Spark version: 3.4.1


In [6]:
from sparknlp.pretrained import ResourceDownloader
ResourceDownloader.showPublicPipelines(lang="en")

+---------------------------------------------------------------------------------------------------------------+------+---------+
| Pipeline                                                                                                      | lang | version |
+---------------------------------------------------------------------------------------------------------------+------+---------+
| dependency_parse                                                                                              |  en  | 2.0.2   |
| check_spelling                                                                                                |  en  | 2.1.0   |
| match_datetime                                                                                                |  en  | 2.1.0   |
| match_pattern                                                                                                 |  en  | 2.1.0   |
| clean_pattern                                                                    

### use the Spark NLP library to load a pre-trained pipeline for English text cleaning and stop word removal

In [7]:
from sparknlp.pretrained import PretrainedPipeline
clean_stop = PretrainedPipeline('clean_stop', lang='en')

clean_stop download started this may take some time.
Approx size to download 13.8 KB
[ | ]clean_stop download started this may take some time.
Approximate size to download 13.8 KB
[ / ]Download done! Loading the resource.
[ — ]

[OK!]


In [8]:
clean_stop.model.stages # clean stop pipeline stages

[document_90b4be8a6e0b,
 SENTENCE_8ba1e4f73af0,
 REGEX_TOKENIZER_8af63c5a17f5,
 STOPWORDS_CLEANER_b5d381c851f5]

### using Apache Spark to read a JSON file containing Yelp review data

In [9]:
review_df = spark.read.json('/kaggle/input/yelp-dataset/review.json')

In [10]:
target_counts = review_df.groupBy('stars').count()
target_counts.show()

+-----+-------+
|stars|  count|
+-----+-------+
|  1.0|1002159|
|  4.0|1468985|
|  3.0| 739280|
|  2.0| 542394|
|  5.0|2933082|
+-----+-------+



### cleaning the Stars column by droping the 3 star review and change 1,2 to zero and 4,5 to one. 
### so we can classify them 

In [11]:
review_df = review_df.filter('stars!=3') 

In [12]:
review_df = review_df.withColumn('target', when((review_df['stars'] == 1) | (review_df['stars'] == 2), 0).otherwise(1))
target_counts = review_df.groupBy('target').count()
target_counts.show()

+------+-------+
|target|  count|
+------+-------+
|     1|4402067|
|     0|1544553|
+------+-------+



In [13]:
review_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- target: integer (nullable = false)



### showing stop word example 

In [14]:
first_text = review_df.select('text').first()[0]

print("First text:", first_text)


First text: Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.


In [15]:
result = clean_stop.annotate(first_text)
result.keys()

dict_keys(['document', 'sentence', 'token', 'cleanTokens'])

In [16]:
' '.join(result['cleanTokens'])

'Total bill horrible service ? $8Gs . crooks actually nerve charge $69 3 pills . checked online pills 19 cents ! Avoid Hospital ERs costs .'

# Clean data 

### 1. lower case the words
### 2. replace all none english words with space 

In [17]:
from pyspark.sql.functions import lower, regexp_replace, trim

# Assuming you already have a SparkSession named 'spark' and a DataFrame named 'review_df'

# Define the PySpark equivalent of your Python code with extra whitespaces removed
review_df = review_df.withColumn('processed_content', trim(lower(regexp_replace('text', '[^a-zA-Z]', ' '))))

# Show the result
review_df.select('text', 'processed_content').show(truncate=True)


+--------------------+--------------------+
|                text|   processed_content|
+--------------------+--------------------+
|Total bill for th...|total bill for th...|
|I *adore* Travis ...|i  adore  travis ...|
|I have to say tha...|i have to say tha...|
|Went in for a lun...|went in for a lun...|
|Today was my seco...|today was my seco...|
|I'll be the first...|i ll be the first...|
|This place has go...|this place has go...|
|I was really look...|i was really look...|
|Like walking back...|like walking back...|
|Walked in around ...|walked in around ...|
|Wow. So surprised...|wow  so surprised...|
|Michael from Red ...|michael from red ...|
|I cannot believe ...|i cannot believe ...|
|You can't really ...|you can t really ...|
|Great lunch today...|great lunch today...|
|We've been a huge...|we ve been a huge...|
|Our family LOVES ...|our family loves ...|
|If you are lookin...|if you are lookin...|
|The food is alway...|the food is alway...|
|Pick any meat on ...|pick any m

### 3. remove extra whitespaces

In [18]:
review_df = review_df.withColumn("processed_content_", regexp_replace(col("processed_content"), " +", " "))

review_df.select('processed_content', 'processed_content_').show(truncate=True)

+--------------------+--------------------+
|   processed_content|  processed_content_|
+--------------------+--------------------+
|total bill for th...|total bill for th...|
|i  adore  travis ...|i adore travis at...|
|i have to say tha...|i have to say tha...|
|went in for a lun...|went in for a lun...|
|today was my seco...|today was my seco...|
|i ll be the first...|i ll be the first...|
|this place has go...|this place has go...|
|i was really look...|i was really look...|
|like walking back...|like walking back...|
|walked in around ...|walked in around ...|
|wow  so surprised...|wow so surprised ...|
|michael from red ...|michael from red ...|
|i cannot believe ...|i cannot believe ...|
|you can t really ...|you can t really ...|
|great lunch today...|great lunch today...|
|we ve been a huge...|we ve been a huge...|
|our family loves ...|our family loves ...|
|if you are lookin...|if you are lookin...|
|the food is alway...|the food is alway...|
|pick any meat on ...|pick any m

### 4. Removing stop words 

In [19]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.pipeline import Pipeline
from pyspark.sql import SparkSession

# Tokenize the text
tokenizer = Tokenizer(inputCol='processed_content', outputCol='words')

# Apply HashingTF to convert words to term frequency vectors
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='raw_features')

# Apply IDF to transform term frequency vectors to TF-IDF vectors
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol='features')

# Define the pipeline
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf])

# Fit and transform the pipeline on the DataFrame
pipeline_model = pipeline.fit(review_df)
transformed_df = pipeline_model.transform(review_df)

# Display the resulting DataFrame
transformed_df.select('processed_content', 'features').show(truncate=False)


24/01/02 18:15:13 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
# Display the resulting DataFrame
transformed_df.select('processed_content', 'features').show(truncate=True)

+--------------------+--------------------+
|   processed_content|            features|
+--------------------+--------------------+
|total bill for th...|(262144,[19036,22...|
|i  adore  travis ...|(262144,[477,568,...|
|i have to say tha...|(262144,[991,9056...|
|went in for a lun...|(262144,[2025,421...|
|today was my seco...|(262144,[1872,230...|
|i ll be the first...|(262144,[1531,232...|
|this place has go...|(262144,[4106,495...|
|i was really look...|(262144,[336,625,...|
|like walking back...|(262144,[2437,254...|
|walked in around ...|(262144,[1619,905...|
|wow  so surprised...|(262144,[3228,421...|
|michael from red ...|(262144,[6498,105...|
|i cannot believe ...|(262144,[2325,454...|
|you can t really ...|(262144,[4959,508...|
|great lunch today...|(262144,[8538,177...|
|we ve been a huge...|(262144,[14390,18...|
|our family loves ...|(262144,[4629,275...|
|if you are lookin...|(262144,[1671,168...|
|the food is alway...|(262144,[27576,30...|
|pick any meat on ...|(262144,[4

24/01/02 18:15:14 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


# training model

### using LogisticRegression to train the model 

In [21]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline



# Check if 'features_vector' column already exists
if 'features_vector' not in transformed_df.columns:
    # VectorAssembler to assemble 'features' into a vector column
    vector_assembler = VectorAssembler(inputCols=['features'], outputCol='features_vector')
    transformed_df = vector_assembler.transform(transformed_df)

# Split the data into training and testing sets
train_ratio = 0.8
test_ratio = 1.0 - train_ratio
seed = 2

train_data, test_data = transformed_df.randomSplit([train_ratio, test_ratio], seed=seed)


# Use Logistic Regression model
lr = LogisticRegression(featuresCol='features_vector', labelCol='target')

# Create a pipeline with the assembler and the logistic regression model
pipeline = Pipeline(stages=[lr])

# Fit the pipeline on the training data
model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol='target')
area_under_curve = evaluator.evaluate(predictions)

print(f"Area under ROC curve: {area_under_curve}")


24/01/02 18:15:19 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:27:49 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:27:52 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/02 18:27:53 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:39:30 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:39:33 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:39:50 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:39:55 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:40:14 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:40:19 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:40:37 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 18:40:41 WARN 

Area under ROC curve: 0.9789038519976963


## evaluating the LogisticRegression model using BinaryClassificationEvaluator

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

# ... (Previous code)

# Assuming 'label_col' is defined as the label column name in your dataset
label_col = 'target'

# Create a BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol=label_col)

# Evaluate additional metrics on the test data
area_under_curve = evaluator.evaluate(predictions)

# Calculate additional metrics manually
true_positive = predictions.filter((col("prediction") == 1) & (col(label_col) == 1)).count()
false_positive = predictions.filter((col("prediction") == 1) & (col(label_col) == 0)).count()
true_negative = predictions.filter((col("prediction") == 0) & (col(label_col) == 0)).count()
false_negative = predictions.filter((col("prediction") == 0) & (col(label_col) == 1)).count()

accuracy = (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f1 = 2 * (precision * recall) / (precision + recall)

# Print the evaluation metrics
print(f"{'Metric':<20}{'Value':>10}")
print(f"{'-'*30}")
print(f"{'Area under ROC curve':<20}{area_under_curve:>10.4f}")
print(f"{'Accuracy':<20}{accuracy:>10.4f}")
print(f"{'Precision':<20}{precision:>10.4f}")
print(f"{'Recall':<20}{recall:>10.4f}")
print(f"{'F1 Score':<20}{f1:>10.4f}")



## Save model

In [26]:
from pyspark.ml import PipelineModel

# ... (Previous code)

# Specify the path where you want to save the model
model_path = "/kaggle/working/logistic_regression_model"

# Save the trained model
model.save(model_path)

# To load the saved model later, you can use the following:
# loaded_model = PipelineModel.load(model_path)

print(f"Model saved to: {model_path}")


24/01/02 20:43:01 WARN TaskSetManager: Stage 284 contains a task of very large size (2097 KiB). The maximum recommended task size is 1000 KiB.


Model saved to: /kaggle/working/logistic_regression_model


### using DecisionTreeClassifier to train the model 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

# Check if 'features_vector' column already exists
if 'features_vector' not in transformed_df.columns:
    # VectorAssembler to assemble 'features' into a vector column
    vector_assembler = VectorAssembler(inputCols=['features'], outputCol='features_vector')
    transformed_df = vector_assembler.transform(transformed_df)

# Split the data into training and testing sets
train_ratio = 0.8
test_ratio = 1.0 - train_ratio
seed = 2

train_data, test_data = transformed_df.randomSplit([train_ratio, test_ratio], seed=seed)

# Assuming you have a binary classification problem
# Use Decision Tree as an example model
dt = DecisionTreeClassifier(featuresCol='features_vector', labelCol='target', maxDepth=5)

# Create a pipeline with the assembler and the decision tree model
pipeline = Pipeline(stages=[dt])

# Fit the pipeline on the training data
model_2 = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model_2.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol='target')
area_under_curve = evaluator.evaluate(predictions)

print(f"Area under ROC curve: {area_under_curve}")


24/01/02 20:44:50 WARN DAGScheduler: Broadcasting large task binary with size 18.7 MiB
24/01/02 20:56:23 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 20:57:03 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
24/01/02 21:08:45 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB
24/01/02 21:22:17 WARN DAGScheduler: Broadcasting large task binary with size 1032.7 KiB
24/01/02 21:22:20 WARN DAGScheduler: Broadcasting large task binary with size 22.4 MiB
24/01/02 21:23:37 WARN MemoryStore: Not enough space to cache rdd_652_2 in memory! (computed 951.0 MiB so far)
24/01/02 21:23:37 WARN BlockManager: Persisting block rdd_652_2 to disk instead.
24/01/02 21:23:41 WARN MemoryStore: Not enough space to cache rdd_652_1 in memory! (computed 1435.3 MiB so far)
24/01/02 21:23:41 WARN BlockManager: Persisting block rdd_652_1 to disk instead.
24/01/02 21:23:42 WARN MemoryStore: Not enough space to cache rdd_652_3 in memory! (compute

## evaluating the DecisionTreeClassifier model using BinaryClassificationEvaluator

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

# ... (Previous code)

# Assuming 'label_col' is defined as the label column name in your dataset
label_col = 'target'

# Create a BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol=label_col)

# Evaluate additional metrics on the test data
area_under_curve = evaluator.evaluate(predictions)

# Calculate additional metrics manually
true_positive = predictions.filter((col("prediction") == 1) & (col(label_col) == 1)).count()
false_positive = predictions.filter((col("prediction") == 1) & (col(label_col) == 0)).count()
true_negative = predictions.filter((col("prediction") == 0) & (col(label_col) == 0)).count()
false_negative = predictions.filter((col("prediction") == 0) & (col(label_col) == 1)).count()

accuracy = (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f1 = 2 * (precision * recall) / (precision + recall)

# Print the evaluation metrics
print(f"{'Metric':<20}{'Value':>10}")
print(f"{'-'*30}")
print(f"{'Area under ROC curve':<20}{area_under_curve:>10.4f}")
print(f"{'Accuracy':<20}{accuracy:>10.4f}")
print(f"{'Precision':<20}{precision:>10.4f}")
print(f"{'Recall':<20}{recall:>10.4f}")
print(f"{'F1 Score':<20}{f1:>10.4f}")


## Save model 

In [ ]:
from pyspark.ml import PipelineModel

# ... (Previous code)

# Specify the path where you want to save the model
model_path = "/kaggle/working/DecisionTreeClassifier_model"

# Save the trained model
model_2.save(model_path)

# To load the saved model later, you can use the following:
# loaded_model = PipelineModel.load(model_path)

print(f"Model saved to: {model_path}")